# **Distribución de créditos por segmento**

In [1]:
# Manejo de datos
import os # Directorios
import pandas as pd # Manipulación df
# Gráficas
import plotly.graph_objects as go #Para obtener librería usar: pip install plotly
from plotly.subplots import make_subplots
import plotly.io as pio # Exportar gráfica

# Obtener el directorio actual de trabajo
directorio_actual = os.getcwd()

# Directorio donde se encuentran los archivos JSON (ruta relativa)
directorio_json = os.path.join(directorio_actual, '../../db/datos_json')

# Obtener la lista de archivos JSON en el directorio
archivos_json = os.listdir(directorio_json)

# Cargar los archivos JSON y crear DataFrames
for archivo in archivos_json:
    nombre_tabla = archivo.replace('datos_', '').replace('.json', '')
    ruta_json = os.path.join(directorio_json, archivo)
    globals()[f"df_{nombre_tabla}"] = pd.read_json(ruta_json)

# Obtener todos los nombres de las variables globales
nombres_variables_globales = list(globals().keys())

# Filtrar los nombres que comienzan con "df_", contienen "alfa_q" y "pachuca"
nombres_df_filtrados = [
    nombre for nombre in nombres_variables_globales 
    # Caso de cuando no son las alfa q
    if nombre.startswith("df_") and "financiamientos" in nombre and "puebla" in nombre

    #if nombre.startswith("df_") and "tulancingo" in nombre    
    #and ("alfa_q" in nombre or "jul_2023" in nombre or "sep_2023" in nombre or "feb_2024" in nombre or "mar_2024" in nombre or "may_2024" in nombre)
]

# Imprimir la lista de DataFrames filtrados
print("Lista de DataFrames filtrados:")
nombres_df_filtrados

Lista de DataFrames filtrados:


['df_financiamientos_2019_puebla',
 'df_financiamientos_2020_puebla',
 'df_financiamientos_2021_puebla',
 'df_financiamientos_2022_puebla',
 'df_financiamientos_2023_puebla',
 'df_financiamientos_2024_puebla']

## 2022

In [2]:
df_financiamientos_2022_pachuca= df_financiamientos_2022_puebla[['id','monto']]

In [3]:
# Definir los rangos de monto de cada segmento
segmentos = {
    "E": (0,  1000000),
    "D": (1000001,  1750000),
    "C": (1750001,  2500000),
    "B": (2500001,  3250000),  
    "A": (3250001,  4000000),
    "S": (4000001, 12000000),
    "L": (12000001, 22000000),
    "ELITE": (22000001, float('inf'))
}

# Crear el DataFrame de segmentos
df_segmentos = pd.DataFrame(segmentos.items(), columns=['segmento', 'rango'])

# Función para asignar segmento
def asignar_segmento(monto):
    for segmento, (limite_inf, limite_sup) in segmentos.items():
        if limite_inf <= monto <= limite_sup:
            return segmento
    return None

# Aplicar la función para asignar segmento a cada fila
df_financiamientos_2022_pachuca['segmento'] = df_financiamientos_2022_pachuca['monto'].apply(asignar_segmento)
df_financiamientos_2022_pachuca.head()


C:\Users\yoe11\AppData\Local\Temp\ipykernel_8452\270888497.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_financiamientos_2022_pachuca['segmento'] = df_financiamientos_2022_pachuca['monto'].apply(asignar_segmento)


,id,monto,segmento
0,1,3898856.45,A
1,2,0.00,E
2,3,0.00,E
3,4,42231.00,E
4,5,81255.00,E


In [4]:
# Calcular cantidad de registros por segmento
creditos_por_segmento = df_financiamientos_2022_pachuca['segmento'].value_counts()
# Calcular el porcentaje de cada segmento
porcentaje_por_segmento = (creditos_por_segmento / len(df_financiamientos_2022_pachuca)) * 100

# Crear DataFrame con los resultados
creditos_segmentos_2022 = pd.DataFrame({
    'Segmento': creditos_por_segmento.index,
    'Cantidad': creditos_por_segmento.values,
    'Porcentaje': porcentaje_por_segmento.values
})

# Asignarle orden a los segmentos
orden_segmentos=["E", "D", "C", "B","A", "S", "L",'ELITE' ]
creditos_segmentos_2022['Segmento'] = pd.Categorical(creditos_segmentos_2022['Segmento'], categories=orden_segmentos, ordered=True)
creditos_segmentos_2022.sort_values('Segmento', inplace=True)

# Ordenar por segmento
creditos_segmentos_2022.sort_values(by='Segmento', inplace=True)

creditos_segmentos_2022

,Segmento,Cantidad,Porcentaje
0,E,7879,64.778426
1,D,2136,17.561457
2,C,892,7.333717
4,B,443,3.642194
5,A,224,1.841651
3,S,523,4.299926
6,L,52,0.427526
7,ELITE,14,0.115103


In [5]:
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6']

# Obtener los porcentajes y los labels en el orden definido
porcentajes = creditos_segmentos_2022['Porcentaje']
labels = creditos_segmentos_2022['Segmento']


fig = go.Figure(data=[go.Pie(labels=labels, values=porcentajes, marker=dict(colors=colores))])
fig.update_traces(textinfo='percent', textposition='inside')
fig.update_layout(
    #title='Distribución de créditos por segmento 2022', # Agregar título
    #showlegend=False  # Ocultar leyenda
    margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)        
)

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo,  carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

guardar_grafico_como_html(fig, 'g_pie_dist_creditos_segmento_2022',  carpeta='assets/graficas')

fig.show()

In [6]:
# Datos proporcionados
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2', '#E50CB6', '#A9A9A9']  # Añadimos gris
porcentajes = creditos_segmentos_2022['Porcentaje']
labels = creditos_segmentos_2022['Segmento']

# Asignar un color gris para los segmentos que no tengan porcentaje
# Crear un diccionario de colores con todos los segmentos, asignando gris a los que no tienen porcentaje
colores_segmentos = {segmento: colores[i] if segmento in labels.values else '#A9A9A9' 
                     for i, segmento in enumerate(segmentos)}

# Ordenar los segmentos según el orden deseado
orden_segmentos = ["E", "D", "C", "B", "A", "S", "L", "ELITE"]

# Crear lista de colores y valores en el orden especificado
colores_ordenados = [colores_segmentos.get(segmento, '#A9A9A9') for segmento in orden_segmentos]
valores_ordenados = [porcentajes[labels == segmento].sum() if segmento in labels.values else 0 for segmento in orden_segmentos]

# Crear gráfico de pastel
fig = go.Figure(data=[go.Pie(
    labels=orden_segmentos, 
    values=valores_ordenados, 
    marker=dict(colors=colores_ordenados)
)])

# Configuración de texto dentro del gráfico
fig.update_traces(
    textinfo='percent+label',  # Mostrar porcentaje y etiquetas
    textposition='inside'     # Ubicar texto dentro de las secciones
)

# Configuración de la leyenda
fig.update_layout(
    margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes
    legend=dict(                  # Personaliza la leyenda
        x=0.5,  # Posición horizontal centrada
        y=1.1,  # Posición vertical ajustada al top
        font=dict(size=18),  # Tamaño de la fuente
        xanchor='center',  # Ancla en el centro horizontal
        yanchor='bottom',  # Ancla en la parte inferior de la leyenda
        orientation='h'  # Cambiar orientación de la leyenda a horizontal
    ), autosize=True,
     xaxis=dict(tickangle=45),  # Ajusta el ángulo de los ticks en el eje X
    yaxis=dict(tickangle=45),   # Ajusta el ángulo de los ticks en el eje Y
    showlegend=True              # Mostrar la leyenda
)

# Función para exportar la gráfica como archivo HTML con configuración personalizada
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Configuración para desactivar herramientas no deseadas
    config = {
        'displaylogo': False,  # Ocultar logo de Plotly
        'modeBarButtonsToRemove': [
            'toImage',       # Botón de guardar como imagen
            'resetScale2d',  # Reset de ejes (no aplica aquí, pero lo incluimos como ejemplo)
            'zoom2d',        # Botón de zoom
            'pan2d',         # Botón de desplazamiento
        ]
    }
    # Exportar el gráfico con configuración
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html', config=config)

# Exportar el gráfico
guardar_grafico_como_html(fig, 'g_pie_dist_creditos_segmento_2022', carpeta='assets/graficas')

# Mostrar el gráfico
fig.show()

## 2023

In [7]:
df_financiamientos_2023_pachuca= df_financiamientos_2023_puebla[['id','monto']]

In [8]:
# Definir los rangos de monto de cada segmento
segmentos = {
    "E": (0,  1000000),
    "D": (1000001,  1750000),
    "C": (1750001,  2500000),
    "B": (2500001,  3250000),  
    "A": (3250001,  4000000),
    "S": (4000001, 12000000),
    "L": (12000001, 22000000),
    "ELITE": (22000001, float('inf'))
}

# Crear el DataFrame de segmentos
df_segmentos = pd.DataFrame(segmentos.items(), columns=['segmento', 'rango'])

# Función para asignar segmento
def asignar_segmento(monto):
    for segmento, (limite_inf, limite_sup) in segmentos.items():
        if limite_inf <= monto <= limite_sup:
            return segmento
    return None

# Aplicar la función para asignar segmento a cada fila
df_financiamientos_2023_pachuca['segmento'] = df_financiamientos_2023_pachuca['monto'].apply(asignar_segmento)
df_financiamientos_2023_pachuca.head()


C:\Users\yoe11\AppData\Local\Temp\ipykernel_8452\777225814.py:24: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,id,monto,segmento
0,1,19817.75,E
1,2,0.00,E
2,3,0.00,E
3,4,33908.00,E
4,5,277298.00,E


In [9]:
# Calcular cantidad de registros por segmento
creditos_por_segmento = df_financiamientos_2023_pachuca['segmento'].value_counts()
# Calcular el porcentaje de cada segmento
porcentaje_por_segmento = (creditos_por_segmento / len(df_financiamientos_2023_pachuca)) * 100

# Crear DataFrame con los resultados
creditos_segmentos_2023 = pd.DataFrame({
    'Segmento': creditos_por_segmento.index,
    'Cantidad': creditos_por_segmento.values,
    'Porcentaje': porcentaje_por_segmento.values
})


# Asignarle orden a los segmentos
orden_segmentos=["E", "D", "C", "B","A", "S", "L",'ELITE' ]
creditos_segmentos_2022['Segmento'] = pd.Categorical(creditos_segmentos_2022['Segmento'], categories=orden_segmentos, ordered=True)
creditos_segmentos_2022.sort_values('Segmento', inplace=True)


# Ordenar por segmento
creditos_segmentos_2023.sort_values(by='Segmento', inplace=True)
creditos_segmentos_2023

,Segmento,Cantidad,Porcentaje
5,A,249,2.297684
4,B,460,4.244717
2,C,930,8.581711
1,D,2089,19.276553
0,E,6481,59.804374
7,ELITE,15,0.138415
6,L,61,0.562886
3,S,552,5.093661


In [10]:
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6']

porcentajes = creditos_segmentos_2023['Porcentaje']
labels = creditos_segmentos_2023['Segmento']

fig = go.Figure(data=[go.Pie(labels=labels, values=porcentajes, marker=dict(colors=colores))])
fig.update_traces(textinfo='percent', textposition='inside')
fig.update_layout(
    #title='Distribución de créditos por segmento 2022', # Agregar título
    #showlegend=False  # Ocultar leyenda
    margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)        
)

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo,  carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

guardar_grafico_como_html(fig, 'g_pie_dist_creditos_segmento_2023',  carpeta='assets/graficas')

fig.show()

In [11]:
# Datos proporcionados
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2', '#E50CB6', '#A9A9A9']  # Añadimos gris

porcentajes = creditos_segmentos_2023['Porcentaje']
labels = creditos_segmentos_2023['Segmento']

# Asignar un color gris para los segmentos que no tengan porcentaje
# Crear un diccionario de colores con todos los segmentos, asignando gris a los que no tienen porcentaje
colores_segmentos = {segmento: colores[i] if segmento in labels.values else '#A9A9A9' 
                     for i, segmento in enumerate(segmentos)}

# Ordenar los segmentos según el orden deseado
orden_segmentos = ["E", "D", "C", "B", "A", "S", "L", "ELITE"]

# Crear lista de colores y valores en el orden especificado
colores_ordenados = [colores_segmentos.get(segmento, '#A9A9A9') for segmento in orden_segmentos]
valores_ordenados = [porcentajes[labels == segmento].sum() if segmento in labels.values else 0 for segmento in orden_segmentos]

# Crear gráfico de pastel
fig = go.Figure(data=[go.Pie(
    labels=orden_segmentos, 
    values=valores_ordenados, 
    marker=dict(colors=colores_ordenados)
)])

# Configuración de texto dentro del gráfico
fig.update_traces(
    textinfo='percent+label',  # Mostrar porcentaje y etiquetas
    textposition='inside'     # Ubicar texto dentro de las secciones
)

# Configuración de la leyenda
fig.update_layout(
    margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes
    legend=dict(                  # Personaliza la leyenda
        x=0.5,  # Posición horizontal centrada
        y=1.1,  # Posición vertical ajustada al top
        font=dict(size=18),  # Tamaño de la fuente
        xanchor='center',  # Ancla en el centro horizontal
        yanchor='bottom',  # Ancla en la parte inferior de la leyenda
        orientation='h'  # Cambiar orientación de la leyenda a horizontal
    ), autosize=True,
    xaxis=dict(tickangle=45),  # Ajusta el ángulo de los ticks en el eje X
    yaxis=dict(tickangle=45),   # Ajusta el ángulo de los ticks en el eje Y
    showlegend=True              # Mostrar la leyenda
)

# Función para exportar la gráfica como archivo HTML con configuración personalizada
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Configuración para desactivar herramientas no deseadas
    config = {
        'displaylogo': False,  # Ocultar logo de Plotly
        'modeBarButtonsToRemove': [
            'toImage',       # Botón de guardar como imagen
            'resetScale2d',  # Reset de ejes (no aplica aquí, pero lo incluimos como ejemplo)
            'zoom2d',        # Botón de zoom
            'pan2d',         # Botón de desplazamiento
        ]
    }
    # Exportar el gráfico con configuración
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html', config=config)

# Exportar el gráfico
guardar_grafico_como_html(fig, 'g_pie_dist_creditos_segmento_2023', carpeta='assets/graficas')

# Mostrar el gráfico
fig.show()

## 2024

In [12]:
df_financiamientos_2024_pachuca= df_financiamientos_2024_puebla[['id','monto']]

In [13]:
# Definir los rangos de monto de cada segmento
segmentos = {
    "E": (0,  1000000),
    "D": (1000001,  1750000),
    "C": (1750001,  2500000),
    "B": (2500001,  3250000),  
    "A": (3250001,  4000000),
    "S": (4000001, 12000000),
    "L": (12000001, 22000000),
    "ELITE": (22000001, float('inf'))
}

# Crear el DataFrame de segmentos
df_segmentos = pd.DataFrame(segmentos.items(), columns=['segmento', 'rango'])

# Función para asignar segmento
def asignar_segmento(monto):
    for segmento, (limite_inf, limite_sup) in segmentos.items():
        if limite_inf <= monto <= limite_sup:
            return segmento
    return None

# Aplicar la función para asignar segmento a cada fila
df_financiamientos_2024_pachuca['segmento'] = df_financiamientos_2024_pachuca['monto'].apply(asignar_segmento)
df_financiamientos_2024_pachuca.head()

C:\Users\yoe11\AppData\Local\Temp\ipykernel_8452\935610015.py:24: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,id,monto,segmento
0,1,8574319.0,S
1,2,1633800.0,D
2,3,0.0,E
3,4,0.0,E
4,5,0.0,E


In [14]:
# Calcular cantidad de registros por segmento
creditos_por_segmento = df_financiamientos_2024_pachuca['segmento'].value_counts()
# Calcular el porcentaje de cada segmento
porcentaje_por_segmento = (creditos_por_segmento / len(df_financiamientos_2024_pachuca)) * 100

# Crear DataFrame con los resultados
creditos_segmentos_2024 = pd.DataFrame({
    'Segmento': creditos_por_segmento.index,
    'Cantidad': creditos_por_segmento.values,
    'Porcentaje': porcentaje_por_segmento.values
})

# Ordenar por segmento
creditos_segmentos_2024.sort_values(by='Segmento', inplace=True)
creditos_segmentos_2024

,Segmento,Cantidad,Porcentaje
5,A,170,2.601775
4,B,302,4.621977
2,C,584,8.937863
1,D,1233,18.870523
0,E,3869,59.213346
7,ELITE,12,0.183655
6,L,37,0.566269
3,S,327,5.004591


In [15]:
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6']

porcentajes = creditos_segmentos_2024['Porcentaje']
labels = creditos_segmentos_2024['Segmento']

fig = go.Figure(data=[go.Pie(labels=labels, values=porcentajes, marker=dict(colors=colores))])
fig.update_traces(textinfo='percent', textposition='inside')
fig.update_layout(
    #title='Distribución de créditos por segmento 2022', # Agregar título
    #showlegend=False  # Ocultar leyenda
    margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)        
)

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo,  carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

guardar_grafico_como_html(fig, 'g_pie_dist_creditos_segmento_2024',  carpeta='assets/graficas')
fig.show()

In [16]:
# Datos proporcionados
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2', '#E50CB6', '#A9A9A9']  # Añadimos gris

porcentajes = creditos_segmentos_2024['Porcentaje']
labels = creditos_segmentos_2024['Segmento']

# Asignar un color gris para los segmentos que no tengan porcentaje
# Crear un diccionario de colores con todos los segmentos, asignando gris a los que no tienen porcentaje
colores_segmentos = {segmento: colores[i] if segmento in labels.values else '#A9A9A9' 
                     for i, segmento in enumerate(segmentos)}

# Ordenar los segmentos según el orden deseado
orden_segmentos = ["E", "D", "C", "B", "A", "S", "L", "ELITE"]

# Crear lista de colores y valores en el orden especificado
colores_ordenados = [colores_segmentos.get(segmento, '#A9A9A9') for segmento in orden_segmentos]
valores_ordenados = [porcentajes[labels == segmento].sum() if segmento in labels.values else 0 for segmento in orden_segmentos]

# Crear gráfico de pastel
fig = go.Figure(data=[go.Pie(
    labels=orden_segmentos, 
    values=valores_ordenados, 
    marker=dict(colors=colores_ordenados)
)])

# Configuración de texto dentro del gráfico
fig.update_traces(
    textinfo='percent+label',  # Mostrar porcentaje y etiquetas
    textposition='inside'     # Ubicar texto dentro de las secciones
)

# Configuración de la leyenda
fig.update_layout(
    margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes
    legend=dict(                  # Personaliza la leyenda
        x=0.5,  # Posición horizontal centrada
        y=1.1,  # Posición vertical ajustada al top
        font=dict(size=18),  # Tamaño de la fuente
        xanchor='center',  # Ancla en el centro horizontal
        yanchor='bottom',  # Ancla en la parte inferior de la leyenda
        orientation='h'  # Cambiar orientación de la leyenda a horizontal
    ),
    xaxis=dict(tickangle=45),  # Ajusta el ángulo de los ticks en el eje X
    yaxis=dict(tickangle=45),   # Ajusta el ángulo de los ticks en el eje Y
    autosize=True,
    showlegend=True  # Mostrar la leyenda
)

# Función para exportar la gráfica como archivo HTML con configuración personalizada
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Configuración para desactivar herramientas no deseadas
    config = {
        'displaylogo': False,  # Ocultar logo de Plotly
        'modeBarButtonsToRemove': [
            'toImage',       # Botón de guardar como imagen
            'resetScale2d',  # Reset de ejes (no aplica aquí, pero lo incluimos como ejemplo)
            'zoom2d',        # Botón de zoom
            'pan2d',         # Botón de desplazamiento
        ]
    }
    # Exportar el gráfico con configuración
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html', config=config)

# Exportar el gráfico
guardar_grafico_como_html(fig, 'g_pie_dist_creditos_segmento_2024', carpeta='assets/graficas')

# Mostrar el gráfico
fig.show()